# Lecture 8 - 29.11.2021 
# Exercise : Pol-InSAR forest height inversion

* DLR's F-SAR acquisition over Traunstein forest (Germany)
* Path: '/projects/data/04-polinsar/'
* SLCs: 
    * Acquisition 1 : slc_15tmpsar0302_L { hh, hv, vv, vh } _t01.rat
    * Acquisition 2 : slc_15tmpsar0302_15tmpsar0303_L { hh, hv, vv, vh } _t01.rat
* Flat-earth: pha_flat_15tmpsar0302_15tmpsar0303_Lhh_t01.rat
* Vertical wavenumber (kz) : kz_2d_demc_15tmpsar0302_15tmpsar0303_t01.rat
* Lidar: Lida_r1503.rat

Objective:
- Estimate forest height using the extremes of the dual-pol Pol-InSAR coherence region and model inversion. 

Tips:
- work on the azimuth - range block [21500 - 4000, 21500 + 6000] - [2300, 4500] ;
- all the needed functions (including the full calculation of the coherence region) have been already implemented.

In [ ]:
# import useful libraries, functions, and modules

import sys
sys.path.append('/projects/src/')

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import filters
from ste_io import *
from tqdm import tqdm

%matplotlib widget



In [ ]:
def calculate_covariance(im1, im2, looksr, looksa) : 
    
    # ... apply definition
    corr = filters.uniform_filter(np.real(im1*np.conj(im2)), [looksa,looksr]) + 1j* \
                filters.uniform_filter(np.imag(im1*np.conj(im2)), [looksa,looksr])
    
    # ... and back to main
    return corr

In [ ]:
def calculate_coh_region(T11, T12, T22, Om_11, Om_12, Om_21, Om_22) : 
    
    # Calculate geometric parameters and extreme coherence of the dual-pol coherence region.
    # Inputs: elements of T matrix and Omega matrix.
    # Outputs: angular extremes (gamma_plus, gamma_minus)
    #          coherence region major axis (a2)
    #          coherence region minor axis (b2)
    #          coherence region orientation (phi)
    #          coherence region center (center)
    
    # calculate the elements of T**(-1/2)
    iT11, iT12, iT21, iT22 = sqrt_inverse(T11, T12, T22)

    # free memory
    del T11, T12, T22

    # whiten ... two 2 x 2 multiplications ==> Matrix P
    aux11, aux12, aux21, aux22 = matmul_2(iT11, iT12, iT21, iT22, Om_11, Om_12, Om_21, Om_22)
    del Om_11, Om_12, Om_21, Om_22
    P11, P12, P21, P22 = matmul_2(aux11, aux12, aux21, aux22, iT11, iT12, iT21, iT22)
    del aux11, aux12, aux21, aux22
    del iT11, iT12, iT21, iT22

    # --- Centering... translate P by 0.5*trace(P) * Identity ==> Matrix O

    center = 0.5 * (P11 + P22)  # it is the center of the coherence region !!
    O11 = P11 - center
    O12 = P12
    O21 = P21
    O22 = P22 - center

    # free memory
    del P11, P12, P21, P22
    
    # ACHTUNG: plus / minus corresponds to the column index of the eigenvectors
    # plus => 1, minus => 2

    # --- Get eigenvalues and eigenvectors of O

    l1, l2, e11, e21, e12, e22 = eigenvalvect_2(O11, O12, O21, O22)
    del l1, l2

    # --- Get r and s (from off-diagonalization of O through the eigenvectors)

    r, s = off_diagonal_2(O11, O12, O21, O22, e11, e21, e12, e22)

    # free memory
    del e11, e21, e12, e22

    # --- finally get geometry

    # ... major axis
    a2 = np.abs(r) + np.abs(s)
    # ... minor axis
    b2 = np.abs(np.abs(r) - np.abs(s))
    # ... orientation
    phi = .5 * (np.angle(r) + np.angle(s))
    
    # --- extremes in phase - gamma_plus / _minus

    gamma_plus = center + a2/2 * np.exp(1j*phi)
    gamma_minus = center - a2/2 * np.exp(1j*phi)
    
    return gamma_plus, gamma_minus, a2, b2, phi, center
    
    

In [ ]:
def make_pauli(slchh, slchv, slcvv, looksr, looksa) :
    
    # 1. Uses function calculate_covariance
    # 2. Convention (rows x columns): inputs are (az x rg), outputs are (rg x az) 
    #                                 for better plotting as #rg <#az
    
    # Calculate T11, T22 and T33
    T11 = calculate_covariance(slchh + slcvv, slchh + slcvv, looksr, looksa)
    T22 = calculate_covariance(slchh - slcvv, slchh - slcvv, looksr, looksa)
    T33 = calculate_covariance(2*slchv, 2*slchv, looksr, looksa)
    
    # make the pauli rgb (+ tranpose, clipping, normalization)
    dimaz = T11.shape[0]
    dimrg = T11.shape[1]
    rgb_pauli = np.zeros((dimrg, dimaz, 3), 'float32')
    rgb_pauli[:, :, 0] = np.transpose( np.clip(np.sqrt(np.abs(T22)), 0, 2.5*np.mean(np.sqrt(np.abs(T22)))) )    # red
    rgb_pauli[:, :, 1] = np.transpose( np.clip(np.sqrt(np.abs(T33)), 0, 2.5*np.mean(np.sqrt(np.abs(T33)))) )    # green
    rgb_pauli[:, :, 2] = np.transpose( np.clip(np.sqrt(np.abs(T11)), 0, 2.5*np.mean(np.sqrt(np.abs(T11)))) )    # blue
    rgb_pauli[:, :, 0] = rgb_pauli[:, :, 0] / np.max(rgb_pauli[:, :, 0])     # red
    rgb_pauli[:, :, 1] = rgb_pauli[:, :, 1] / np.max(rgb_pauli[:, :, 1])     # green
    rgb_pauli[:, :, 2] = rgb_pauli[:, :, 2] / np.max(rgb_pauli[:, :, 2])     # blue
    
    # ... and back to main
    return rgb_pauli

In [ ]:
def matmul_2(a11, a12, a21, a22, b11, b12, b21, b22) :
    
    # Implements the multiplication A.B of two 2 x 2 matrices A and B.
    # Inputs are the elements of matrices (row-wise). 
    # Outputs are the elements of the matrix product (row-wise).
    
    c11 = a11*b11 + a12*b21
    c12 = a11*b12 + a12*b22
    c21 = a21*b11 + a22*b21 
    c22 = a21*b12 + a22*b22
    
    # ... and back to main
    return c11, c12, c21, c22

In [ ]:
def eigenvalvect_2(a11, a12, a21, a22) :
    
    # Calculates eigenvalues / eigenvectors of a generic complex-valued 2 x 2 matrix.
    # Inputs are the matrix elements (row-wise) - NumPy arrays
    # Outputs are the eigenvalues (l1, l2), and the eigenvector (E1, E2) elements - NumPy arrays
    #                                       E1 = [e11, e21] , U2 = [e12, e22]
    
    # eigenvalues 
    l1 = 0.5 * ((a11+a22) + np.sqrt( (a11-a22)**2 + 4*a12*a21 )) 
    l2 = 0.5 * ((a11+a22) - np.sqrt( (a11-a22)**2 + 4*a12*a21 ))
    
    # eigenvector 1
    e11 = -a12
    e21 = a11 - l1
    # normalize eigenvector 1
    norm_ = np.sqrt( np.abs(e11)**2 + np.abs(e21)**2 )
    e11 = e11 / norm_
    e21 = e21 / norm_
    # eigenvector 2
    e12 = a22 - l2
    e22 = -a21
    # normalize eigenvector 2
    norm_ = np.sqrt( np.abs(e12)**2 + np.abs(e22)**2 )
    e12 = e12 / norm_
    e22 = e22 / norm_
    
    del norm_
    
    # ... and back to main
    return l1, l2, e11, e21, e12, e22

In [ ]:
def off_diagonal_2(a11, a12, a21, a22, e11, e21, e12, e22) :

    # Calculates off-diagonal form a 2 x 2 matrix A given its eigenvalues in E
    # Inputs are the matrix elements (row-wise) - NumPy arrays
    #        and the eigenvector (E1, E2) elements - NumPy arrays
    #                                       E1 = [e11, e21] , U2 = [e12, e22]
    # Outputs are the off-diagonal elements: o12, o21
    
    # rotation angle theta
    theta = np.angle( np.conj(e11)*e12 + np.conj(e21)*e22  )
    
    # first column of the off-diagonalizing matrix (normalized)
    u11 = np.exp(1j*theta)*e11 + e12
    u21 = np.exp(1j*theta)*e21 + e22
    norm_ = np.sqrt( np.abs(u11)**2 + np.abs(u21)**2 )
    u11 = u11 / norm_
    u21 = u21 / norm_
    
    # second column of the off-diagonalizing matrix (normalized)
    u12 = np.exp(1j*theta)*e11 - e12
    u22 = np.exp(1j*theta)*e21 - e22
    norm_ = np.sqrt( np.abs(u12)**2 + np.abs(u22)**2 )
    u12 = u12 / norm_
    u22 = u22 / norm_
   
    del norm_
    
    # calculate off-diagonal elements
    o12 = a11 * ( np.conj(u11)*u12 - np.conj(u21)*u22 ) +\
          a12 * np.conj(u11) * u22 +\
          a21 * np.conj(u21) * u12
    o21 = a11 * ( np.conj(u12)*u11 - np.conj(u22)*u21 ) +\
          a12 * np.conj(u12) * u21 +\
          a21 * np.conj(u22) * u11
    
    # ... and back to main
    return o12, o21

In [ ]:
def sqrt_inverse(a11, a12, a22) :

    # Given a 2 x 2 Hermitian matrix A, calculates A**(-1/2) using eigendecomposition
    # Inputs are the upper-diagonal matrix elements (row-wise) - NumPy arrays.
    # Outputs are the elements of the sqrt inverse matrix.
    # Uses function matmul_2.
    
    # Calculate eigenvalues (l1, l2)
    sigma = 0.5 * (a11 + a22)
    D = a11*a22 - a12*np.conj(a12)
    delta = np.sqrt(sigma**2 - D)
    l1 = sigma + delta
    l2 = sigma - delta
    
    # Calculate eigenvector matrix elements (eigenvectors are the columns)
    norm_ = np.sqrt( a12*np.conj(a12) + ((a11-a22)/2 - delta)**2 )
    v11 = -a12 / norm_
    v21 = ((a11-a22)/2 - delta) / norm_
    v12 = ((a11-a22)/2 - delta) / norm_
    v22 = np.conj(a12) / norm_
    
    del norm_, delta, D, sigma
    
    # calculate sqrt inverse matrix elements
    zz = np.zeros(l1.shape, 'float32')
    aux11, aux12, aux21, aux22 = matmul_2(v11, v12, v21, v22, 1/np.sqrt(l1), zz, zz, 1/np.sqrt(l2))
    del l1, l2
    i11, i12, i21, i22 = matmul_2(aux11, aux12, aux21, aux22, np.conj(v11), np.conj(v21), np.conj(v12), np.conj(v22))
    del aux11, aux12, aux21, aux22
    
    # ... and back to main
    return i11, i12, i21, i22  

**Input parameters**

In [ ]:
# --- Inputs

# path 2 images
path = '/projects/data/04-polinsar/'

# Input pixel spacing, in meters
spacrg = 0.59941552
spacaz = 0.19507939

# Output range resolution, in meters
resrg = 5.
resaz = 5.

# Image block for processing
minrg = 2300
maxrg = 4500
minaz = 21500 - 4000
maxaz = 21500 + 6000


In [ ]:
# --- Calculate number of looks

looksr = int( resrg / spacrg )
if looksr % 2 == 0 : looksr = looksr +1
looksa = int( resaz / spacaz )
if looksa % 2 == 0 : looksa = looksa +1


**Step 1: Open images, and visualize a Pauli**

In [ ]:
# --- Open images

# all channels for the first acquisition to make the the Pauli
slchh1 = rrat(path + 'slc_15tmpsar0302_Lhh_t01.rat', block = [minaz, maxaz+1, minrg, maxrg+1])
slchv1 = rrat(path + 'slc_15tmpsar0302_Lhv_t01.rat', block = [minaz, maxaz+1, minrg, maxrg+1])
slcvv1 = rrat(path + 'slc_15tmpsar0302_Lvv_t01.rat', block = [minaz, maxaz+1, minrg, maxrg+1])

# only relevant channels for the second acquisition
slchh2 = rrat(path + 'slc_coreg_15tmpsar0302_15tmpsar0303_Lhh_t01.rat', block = [minaz, maxaz+1, minrg, maxrg+1])
slchv2 = rrat(path + 'slc_coreg_15tmpsar0302_15tmpsar0303_Lhv_t01.rat', block = [minaz, maxaz+1, minrg, maxrg+1])
slcvv2 = rrat(path + 'slc_coreg_15tmpsar0302_15tmpsar0303_Lvv_t01.rat', block = [minaz, maxaz+1, minrg, maxrg+1])


**Step 2 : Compensate flat-earth**

In [ ]:
# --- Open flat-earth phase
fe = rrat(path + 'pha_flat_15tmpsar0302_15tmpsar0303_Lhh_t01.rat', block = [minaz, maxaz+1, minrg, maxrg+1])

# --- Compensate
slchh2 = slchh2 * np.exp(1j * fe)
slchv2 = slchv2 * np.exp(1j * fe)
slcvv2 = slcvv2 * np.exp(1j * fe)

# free memory
del fe


**Step 3 : Calculate the elements of T1, T2, and Omega matrices**

In [ ]:
# --- create channels
im1_1 = slchh1 - slcvv1
im2_1 = 2*slchv1
im1_2 = slchh2 - slcvv2
im2_2 = 2*slchv2 

# free memory
del slchh1, slchv1, slcvv1, slchh2, slchv2, slcvv2

# --- T1 
T1_11 = calculate_covariance(im1_1, im1_1, looksr, looksa)
T1_12 = calculate_covariance(im1_1, im2_1, looksr, looksa)
T1_22 = calculate_covariance(im2_1, im2_1, looksr, looksa)

# --- T2 
T2_11 = calculate_covariance(im1_2, im1_2, looksr, looksa)
T2_12 = calculate_covariance(im1_2, im2_2, looksr, looksa)
T2_22 = calculate_covariance(im2_2, im2_2, looksr, looksa)

# --- Omega
Om_11 = calculate_covariance(im1_1, im1_2, looksr, looksa)
Om_12 = calculate_covariance(im1_1, im2_2, looksr, looksa)
Om_21 = calculate_covariance(im2_1, im1_2, looksr, looksa)
Om_22 = calculate_covariance(im2_1, im2_2, looksr, looksa)

# free memory
del im1_1, im2_1, im1_2, im2_2

# --- polarization 1 and polarization 2 InSAR coherences
gamma1 = Om_11 / np.sqrt(T1_11) / np.sqrt(T2_11)
gamma2 = Om_22 / np.sqrt(T1_22) / np.sqrt(T2_22)


**Step 4 : Get coherence region parameters (gamma_plus, gamma_minus, geometry)**

In [ ]:
# --- calculate T

T11 = 0.5 * (T1_11 + T2_11)
T12 = 0.5 * (T1_12 + T2_12)
T22 = 0.5 * (T1_22 + T2_22)

# free memory
del T1_11, T1_12, T1_22, T2_11, T2_12, T2_22

# --- compute coherence region paramters
gamma_plus, gamma_minus, a2, b2, phi, center = calculate_coh_region(T11, T12, T22, Om_11, Om_12, Om_21, Om_22)


**Step 5 : Find the volume-only coherence**

**Step 6 : Calculate ground phase**

In [ ]:
v = gamma_g - gamma_v
x = -np.real( np.conj(v)*gamma_v ) / np.abs(v)**2 + \
            np.sqrt( np.real(np.conj(v)*gamma_v)**2 - (np.abs(gamma_v)**2 - 1)* np.abs(v)**2 ) / np.abs(v)**2
cohg = gamma_v + v*x
phig  = np.angle(cohg)



**Step 7 : Visualize coherence region in 1 pixel**

In [ ]:
# --- open lidar height

Hl = rrat(path + 'Lida_r1503.rat', block = [minaz, maxaz+1, minrg, maxrg+1])

# --- fix rg-az coordinates

azp = 276
rgp = 1453

# --- calculate border of coherence region
# ...

# - now plot

# ... prepare complex plane
plt.figure( figsize = (7,7) )
for nn in range(6) : plt.plot(nn*.2*np.cos(theta), nn*.2*np.sin(theta), color = 'b', lw = 0.5)
plt.plot([-1, 1], [0, 0], color = 'b', linestyle = '--', lw = 0.5)
plt.plot([0, 0], [-1, 1],  color = 'b', linestyle = '--', lw = 0.5)
plt.xlabel('Real part')
plt.ylabel('Imaginary part')

# ... plot coherence region before compensating phig
# ...

# plot coherence region after compensating phig
# ...


**Step 8 : Compensate ground phase** 

**Step 9 : Plot look-up table for the chosen pixel**

In [ ]:
# --- load incidence angle

incang = rrat(path + 'incidence_15tmpsar0302_L_t01.rat', block = [minaz, maxaz+1, minrg, maxrg+1])

# --- Define interval of variation of height and extinction

H = np.linspace(0, 50, 51)  # height
# s = np.linspace(0, 2, 31)   # extinction
s = np.tan(np.linspace(0.000001, .99, 31) * np.pi/2) / 30 

# make meshgrids
Hmat = np.outer(np.ones(s.size, 'float32'), H)
smat = np.outer(s, np.ones(H.size, 'float32'))

# --- calculate LUT = look-up table = all the coherences for every combination of height and extinction
p1 = 2*smat/incang[azp, rgp]
p2 = 2*smat/incang[azp, rgp] + 1j*kz[azp, rgp]
lut = ( p1 * (np.exp(p2*Hmat)-1) ) / ( p2 * (np.exp(p1*Hmat)-1))
lut[Hmat == 0] = 1
HH = Hmat[np.logical_and(smat < 1e-6, Hmat !=0)]
lut[np.logical_and(smat < 1e-6, Hmat !=0)] = \
        np.sinc(kz[azp, rgp]*Hmat[np.logical_and(smat < 1e-6, Hmat !=0)]/2/np.pi) * \
                        np.exp(1j*kz[azp, rgp]*Hmat[np.logical_and(smat < 1e-6, Hmat !=0)]/2 )

# ... display
# ... prepare complex plane
# ...



**Step 10 : Forest height inversion & validation**

In [ ]:
# --- process  

In [ ]:
# --- display estimates


In [ ]:
# --- validation

